In [1]:
import sys, os
from pathlib import Path

BASE_DIR = Path(os.path.dirname(os.getcwd()))
sys.path.append(str(BASE_DIR))

In [2]:
from extractors.text_extractors.german_text_extractor import GermanTextEmbeddingExtractor

embedding_extractor = GermanTextEmbeddingExtractor()

/home/amin/amin/University/Term8/NLP/project/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
SPLIT = "test"

In [4]:
import torch

features_path = BASE_DIR / "data/saved_features"
real_indexes = torch.load(features_path / f"indexes_{SPLIT}.pt").tolist()
print(len(real_indexes))

with open(BASE_DIR / "data" / "texts" / "german" / f"{SPLIT}.txt") as f:
    all_sentences = [l.strip() for l in f.readlines()]
    sentences = [all_sentences[i] for i in real_indexes]
# sentences = sentences[:n]
print(len(sentences))
with open(BASE_DIR / "data" / "labels" / f"sentiment_{SPLIT}.txt") as f:
    all_sentiments = [int(l.strip()) for l in f.readlines()]
    sentiments = [all_sentiments[i] for i in real_indexes]
# sentiments = sentiments[:n]
print(len(sentiments))

3331
3331
3331


In [5]:
results = embedding_extractor.extract_embedding(sentences)

RuntimeError: CUDA out of memory. Tried to allocate 284.00 MiB (GPU 0; 1.96 GiB total capacity; 1.24 GiB already allocated; 225.38 MiB free; 1.29 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [11]:
import evaluate


def convert_label(label):
    if label == "NEU":
        return 0
    elif label == "NEG":
        return 1
    elif label == "POS":
        return 2


accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
precision_macro = evaluate.load("precision")
precision_micro = evaluate.load("precision")
precision_weighted = evaluate.load("precision")
f1 = evaluate.load("f1")
f1_macro = evaluate.load("f1")
f1_micro = evaluate.load("f1")
f1_weighted = evaluate.load("f1")
recall = evaluate.load("recall")
recall_macro = evaluate.load("recall")
recall_micro = evaluate.load("recall")
recall_weighted = evaluate.load("recall")

for index, result in enumerate(results):
    predicted_label = convert_label(result["label"])
    accuracy.add(predictions=predicted_label, references=sentiments[index])
    precision.add(predictions=predicted_label, references=sentiments[index])
    precision_macro.add(predictions=predicted_label, references=sentiments[index])
    precision_micro.add(predictions=predicted_label, references=sentiments[index])
    precision_weighted.add(predictions=predicted_label, references=sentiments[index])
    f1.add(predictions=predicted_label, references=sentiments[index])
    f1_macro.add(predictions=predicted_label, references=sentiments[index])
    f1_micro.add(predictions=predicted_label, references=sentiments[index])
    f1_weighted.add(predictions=predicted_label, references=sentiments[index])
    recall.add(predictions=predicted_label, references=sentiments[index])
    recall_macro.add(predictions=predicted_label, references=sentiments[index])
    recall_micro.add(predictions=predicted_label, references=sentiments[index])
    recall_weighted.add(predictions=predicted_label, references=sentiments[index])

print(accuracy.compute())
print(precision.compute(average=None))
print(precision_macro.compute(average="macro"))
print(precision_micro.compute(average="micro"))
print(precision_weighted.compute(average="weighted"))
print(f1.compute(average=None))
print(f1_macro.compute(average="macro"))
print(f1_micro.compute(average="micro"))
print(f1_weighted.compute(average="weighted"))
print(recall.compute(average=None))
print(recall_macro.compute(average="macro"))
print(recall_micro.compute(average="micro"))
print(recall_weighted.compute(average="weighted"))

{'accuracy': 0.42389672770939657}
{'precision': array([0.30227828, 0.77892919, 0.7414966 ])}
{'precision': 0.6075680206384703}
{'precision': 0.42389672770939657}
{'precision': 0.6460379450717734}
{'f1': array([0.44921403, 0.4382896 , 0.33641975])}
{'f1': 0.4079744604145013}
{'f1': 0.42389672770939657}
{'f1': 0.41043375439932483}
{'recall': array([0.87411765, 0.30493577, 0.21756487])}
{'recall': 0.4655394282429057}
{'recall': 0.42389672770939657}
{'recall': 0.42389672770939657}
